WBCD Dataset Neural Network 

### Import Libraries

In [ ]:
import mlrose

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

### Import more Libraries


In [2]:
import numpy as np
import pandas as pd
import time 

import matplotlib.pyplot as plt

# from sklearn.cross_validation import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from textwrap import wrap

### Load Dataset

In [3]:
RANDOM_SEED = 25

df = pd.read_csv('../datasets/us_income/adult-train.csv', delimiter=',', quotechar='"')


### Dataframe without Preprocessing 

In [4]:
print("Printing dataframe head (without any preprocessing)....")
print(df.head(10))

Printing dataframe head (without any preprocessing)....
   Age     Workclass  Fnlwgt      Education  EducationNum  \
0   34   Federal-gov   67083      Bachelors            13   
1   72   Federal-gov   39110           11th             7   
2   45   Federal-gov  235891     Assoc-acdm            12   
3   26   Federal-gov  206983    Prof-school            15   
4   31   Federal-gov  139455      Bachelors            13   
5   27   Federal-gov  196386     Assoc-acdm            12   
6   19   Federal-gov  255921   Some-college            10   
7   60   Federal-gov   27466   Some-college            10   
8   59   Federal-gov  212448        HS-grad             9   
9   32   Federal-gov  148138      Bachelors            13   

         MaritalStatus          Occupation    Relationship  \
0        Never-married     Exec-managerial       Unmarried   
1             Divorced        Adm-clerical   Not-in-family   
2        Never-married        Adm-clerical   Not-in-family   
3   Married-civ-spouse  

### Dataset Preprocessing 

In [5]:
df.replace(' <=50K', 0, inplace=True)
df.replace(' >50K', 1, inplace=True)

df = pd.get_dummies(df)
# print(df.head(5))

X = df.loc[:, df.columns != 'Income']
y = df['Income']

### Sanity Check on Dataframe 

In [6]:
print("Sanity Check: Printing out dataframe and shape after preprocessing... ")
print(df.head(10))
print("df.shape: ", df.shape)


Sanity Check: Printing out dataframe and shape after preprocessing... 
   Age  Fnlwgt  EducationNum  CapitalGain  CapitalLoss  HoursPerWeek  Income  \
0   34   67083            13         1471            0            40       0   
1   72   39110             7            0            0             8       0   
2   45  235891            12            0            0            40       0   
3   26  206983            15            0            0            60       0   
4   31  139455            13            0            0            40       0   
5   27  196386            12         4064            0            40       0   
6   19  255921            10            0            0            20       0   
7   60   27466            10            0            0            40       0   
8   59  212448             9            0            0            40       0   
9   32  148138            13            0         2002            40       0   

   Workclass_ Federal-gov  Workclass_ Local-gov 

### Training and Testing Split, Scaling 

In [7]:
# Split into 30%  training data, 70% testing data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30, random_state=RANDOM_SEED)

# Apply scaling. Large values of certain features undesireable for NN
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

/home/saksham/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/saksham/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  
/home/saksham/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


### Sanity Check on X_train

In [8]:
print("Sanity Check: Printing out X_train... ")
print(X_train[:4])
print("X_train.shape: ", X_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape: ", y_test.shape)


Sanity Check: Printing out X_train... 
[[ 0.80094155 -1.53315845 -0.05190318 -0.1485495  -0.21797847  0.33651868
  -0.17935488 -0.27366136  0.59593187 -0.19138411 -0.30025548 -0.21174501
  -0.01686019 -0.16777087 -0.18681609 -0.11231166 -0.06898521 -0.09729794
  -0.13914294 -0.12346382 -0.18708745 -0.2144172  -0.44821724 -0.11487839
  -0.69458353 -0.23979288 -0.04016192 -0.13664626  1.88030667 -0.40388429
  -0.02482164  1.07637017 -0.11338783 -0.69113933 -0.18173254 -0.16955304
  -0.37574134 -0.01686019 -0.39158358  2.5674644  -0.1839458  -0.21514122
  -0.26362857 -0.34588621 -0.07170518 -0.39527605 -0.14990271 -0.3660739
  -0.17709161 -0.23354385  1.19881045 -0.59126212 -0.17609341 -0.41516053
  -0.34597203 -0.22121665 -0.09729794 -0.17694931 -0.32043606 -0.08847901
   0.40451868 -0.69608262  0.69608262 -0.02176847 -0.06089422 -0.04920795
  -0.04016192 -0.05016592 -0.04723434 -0.03155373 -0.05684339 -0.05293692
  -0.03373476 -0.06244202 -0.03079256 -0.04356792 -0.03644116  0.
  -0.021

### Misc NN Testing (Immediate output for single set of specifications)

In [16]:
np.random.seed(5)

hidden_node_architecture = [15, 15, 15]

# Lists to hold values for x and y axes 
max_iter_list = list()
testError = []
trainError = []

# Create list to hold data on each trial
data = []
# Columns for df we'll create after loop 
cols = ["Max Iterations", "training accuracy", "testing accuracy", "training MSE", "testing MSE"]

# Make decay object for Simulated Annealing to work 
decayObject = mlrose.ArithDecay()

# COMMENCE TRAINING LOOP 
for max_iter in range(500, 10001, 500) :
    
    print("Training NN with max_iter: ", max_iter)
    
    nn_sigmoid = mlrose.NeuralNetwork(hidden_nodes = hidden_node_architecture, activation = 'sigmoid', 
                                        algorithm = 'simulated_annealing', 
                                        max_iters = max_iter, bias = True, is_classifier = True, 
                                        learning_rate = 0.01, early_stopping = True, 
                                        schedule=decayObject, 
                                        clip_max = 5, max_attempts = 100)

    nn_sigmoid.fit(X_train, y_train)
    
    # Prediction based on y_train and y_test 
    y_train_pred = nn_sigmoid.predict(X_train)
    y_test_pred = nn_sigmoid.predict(X_test)
    
    # MSE Values 
    train_err = mean_squared_error(y_train,
                        y_train_pred)
    test_err = mean_squared_error(y_test,
                        y_test_pred) 

    # Accuracy Values
    y_train_accuracy = accuracy_score(y_train, y_train_pred)
    y_test_accuracy = accuracy_score(y_test, y_test_pred)
    
    data.append([max_iter, y_train_accuracy, y_test_accuracy, train_err, test_err])
    
    
    # Append data to lists
    testError.append(test_err)
    trainError.append(train_err)
    max_iter_list.append(max_iter)
    
    
# Store results from above into df 
result_df = pd.DataFrame(data, columns=cols)
print(result_df)
    


Time taken to train NN:  24.95978093147278

Misc Network Statistics

y_train_accuracy:  0.7479751811680008 %
y_test_accuracy: 0.7583158360039783 %

train_err (MSE):  0.25202481883199923
test_err: (MSE) 0.24168416399602166


In [ ]:
plt.figure()
title = "Cesus NN Simulated Annealing (Arithmetic Decay) : Error x Max Iterations - Learning Rate = .01, Hidden Layers = [15,15,15]"
plt.title('\n'.join(wrap(title,60)))
# plt.subplots_adjust(top=0.85)
plt.plot(max_iter_list, testError, '-', label='Test Error')
plt.plot(max_iter_list, trainError, '-', label='Train Error')
plt.legend()
plt.xlabel('Max Iterations')
plt.ylabel('Error')
filename = 'WBCD_SA(ArithDecay)_MaxIter.png'
plt.savefig("../plots/" + filename)

### Train Neural Network (RHC, Sigmoid)

In [ ]:
# Initialize neural network object and fit object - attempt 1

hidden_node_architecture = list()

for x in range(5, 40, 5):
    for y in range(5, 40, 5):
        for z in range(5, 40, 5):.
            hidden_node_architecture.append([x,y,z])

np.random.seed(5)

lr_nn_model_RHC_sigmoid = mlrose.NeuralNetwork(hidden_nodes = [], activation = 'sigmoid', 
                                    algorithm = 'random_hill_climb', 
                                    max_iters = 1000, bias = True, is_classifier = True, 
                                    learning_rate = 0.01, early_stopping = True, 
                                    clip_max = 5, max_attempts = 100)

train_start_time = time.time() 

lr_nn_model_RHC_sigmoid.fit(X_train, y_train)

train_end_time = time.time()
print("Time taken to train NN: ", train_end_time - train_start_time)

### Accuracy, MSE (RHC, Sigmoid)


In [ ]:
# Prediction based on y_train and y_test 
y_train_pred = lr_nn_model_RHC_sigmoid.predict(X_train)
y_test_pred = lr_nn_model_RHC_sigmoid.predict(X_test)


# MSE Values 
train_err = mean_squared_error(y_train,
                    y_train_pred)
test_err = mean_squared_error(y_test,
                    y_test_pred) 

# Accuracy Values
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)



# Print everything out

print("")

print("RHC, Sigmoid Statistics")

print("")

print("y_train_accuracy: ", y_train_accuracy, "%")
print("y_test_accuracy:", y_test_accuracy, "%")

print ("")

print("train_err (MSE): ", train_err)
print("test_err: (MSE)", test_err)

### Train Neural Network (RHC, Relu)

In [ ]:
# Initialize neural network object and fit object - attempt 1

np.random.seed(5)

lr_nn_model_RHC_Relu = mlrose.NeuralNetwork(hidden_nodes = [], activation = 'relu', 
                                    algorithm = 'random_hill_climb', 
                                    max_iters = 1000, bias = True, is_classifier = True, 
                                    learning_rate = 0.01, early_stopping = True, 
                                    clip_max = 5, max_attempts = 100)

lr_nn_model_RHC_Relu.fit(X_train, y_train)

### Accuracy, MSE (RHC, Relu)

In [ ]:
# Prediction based on y_train and y_test 
y_train_pred = lr_nn_model_RHC_Relu.predict(X_train)
y_test_pred = lr_nn_model_RHC_Relu.predict(X_test)


# MSE Values 
train_err = mean_squared_error(y_train,
                    y_train_pred)
test_err = mean_squared_error(y_test,
                    y_test_pred) 

# Accuracy Values
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)


# Print everything out

print("")

print("RHC, Relu Statistics")

print("")

print("y_train_accuracy: ", y_train_accuracy, "%")
print("y_test_accuracy:", y_test_accuracy, "%")

print ("")

print("train_err (MSE): ", train_err)
print("test_err: (MSE)", test_err)

### Train Neural Network (Gradient Descent)

In [ ]:
# Initialize neural network object and fit object - attempt 1

np.random.seed(5)

lr_nn_model_GradientDescent = mlrose.NeuralNetwork(hidden_nodes = [30, 30, 30], activation = 'relu', 
                                    algorithm = 'gradient_descent', 
                                    max_iters = 1000, bias = True, is_classifier = True, 
                                    learning_rate = 0.1, early_stopping = True, 
                                    clip_max = 5, max_attempts = 100)

train_start_time = time.time() 

lr_nn_model_GradientDescent.fit(X_train, y_train)

train_end_time = time.time()
print("Time taken to train NN: ", train_end_time - train_start_time)




### Test Benchmark Accuracy 

In [ ]:
# Prediction based on y_train and y_test 
y_train_pred = lr_nn_model_RHC_Relu.predict(X_train)
y_test_pred = lr_nn_model_RHC_Relu.predict(X_test)


# MSE Values 
train_err = mean_squared_error(y_train,
                    y_train_pred)
test_err = mean_squared_error(y_test,
                    y_test_pred) 

# Accuracy Values
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)


# Print everything out

print("")

print("RHC, Relu Statistics")

print("")

print("y_train_accuracy: ", y_train_accuracy, "%")
print("y_test_accuracy:", y_test_accuracy, "%")

print ("")

print("train_err (MSE): ", train_err)
print("test_err: (MSE)", test_err)